# Solving a complex task with a multi-agent hierarchy

This notebook is part of the [Hugging Face Agents Course](https://www.hf.co/learn/agents-course), a free Course from beginner to expert, where you learn to build Agents.

![Agents course share](https://huggingface.co/datasets/agents-course/course-images/resolve/main/en/communication/share.png)

The reception is approaching! With your help, Alfred is now nearly finished with the preparations.

But now there's a problem: the Batmobile has disappeared. Alfred needs to find a replacement, and find it quickly.

Fortunately, a few biopics have been done on Bruce Wayne's life, so maybe Alfred could get a car left behind on one of the movie set, and re-engineer it up to modern standards, which certainly would include a full self-driving option.

But this could be anywhere in the filming locations around the world - which could be numerous.

So Alfred wants your help. Could you build an agent able to solve this task?

> 👉 Find all Batman filming locations in the world, calculate the time to transfer via boat to there, and represent them on a map, with a color varying by boat transfer time. Also represent some supercar factories with the same boat transfer time.

Let's build this!

In [ ]:
# !pip install 'smolagents[litellm]' matplotlib geopandas shapely kaleido -q

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
# We first make a tool to get the cargo plane transfer time.
import math
from typing import Optional, Tuple

from smolagents import tool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


For the model provider, we use Together AI, one of the new [inference providers on the Hub](https://huggingface.co/blog/inference-providers)!

Regarding the GoogleSearchTool: this requires either having setup env variable `SERPAPI_API_KEY` and passing `provider="serpapi"` or having `SERPER_API_KEY` and passing `provider=serper`.

If you don't have any Serp API provider setup, you can use `DuckDuckGoSearchTool` but beware that it has a rate limit.

In [3]:
import os
from PIL import Image
from smolagents import CodeAgent, GoogleSearchTool, HfApiModel, VisitWebpageTool


model = HfApiModel(model_id="Qwen/Qwen2.5-Coder-32B-Instruct", provider="together")

In [4]:
task = """Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time."""

In [5]:
from dotenv import load_dotenv, find_dotenv
import os

load_dotenv(find_dotenv())

os.environ["SERPAPI_API_KEY"] = os.getenv('SERPAPI_API_KEY')

In [6]:
agent = CodeAgent(
    model=model,
    tools=[GoogleSearchTool(), VisitWebpageTool(), calculate_cargo_travel_time],
    additional_authorized_imports=["pandas"],
    max_steps=20,
)

In [7]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations = web_search(query="Batman filming locations")                                                  
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results
0. [The Batman (2022) - Filming & production](https://www.imdb.com/title/tt1877830/locations/)
Source: IMDb

Filming locations: Necropolis Cemetery, Glasgow, Scotland, UK (Batman and Selina leaving the cemetery) Helpful•86 1
St. George's Hall, Liverpool, England, UK

1. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022-2.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

2. [Where was The Batman filmed? ALL the Filming Locations 
...](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
Source: Atlas of Wonders

Guide to the all the filming locations of The Batman - The 2022 movie with Robert Pattison filmed in Chicago and 
The UK - The City and the Wayne House.

3. [Batman (1989) - Filming & production](https://www.imdb.com/title/tt0096895/locations/)
Source: IMDb

Filming locations: Knebworth House, Knebworth, Hertfordshire, England, UK (Wayne Manor; exterior) Helpful•21 0 
Acton Lane Power Station, Acton Lane, Acton, ...

4. [Dark Knight Rises Tour: See Batman Movie 
Locations](https://www.travelchannel.com/interests/arts-and-culture/photos/see-batman-movie-locations)
Source: Travel Channel

See Batman Movie Locations · Wollaton Hall · Carnegie Mellon University · The Farmiloe Building · Queensboro Bridge
· Wall Street · Mehrangarh Fort · Turda ...

5. [12 Batman Movie Locations You Can 
Visit!](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Date published: Jan 10, 2023
Source: Travel and Leisure Asia

Here are 12 real-world Batman movie locations that you can visit now · St. George's Hall, Liverpool · Two Temple 
Place, London · Wollaton Hall, ...

6. [The Batman | Film Locations](https://movie-locations.com/movies/b/The-Batman-2022.php)
Source: The Worldwide Guide To Movie Locations

Film locations for The Batman (2022) in Liverpool, London, Glasgow and Chicago.

7. [Batman Movies and Filming Locations in Los 
Angeles](https://www.traveltodayla.com/post/batman-movies-in-la?srsltid=AfmBOooorcSbW3C2fADdMD1YYdvUC3c2NF0SbLACWxE
7SjnrtK-fggFB)
Source: LA Today

Westwood, Los Angeles, CA. Step into the world of Batman and Robin with a visit to the Kirkeby Center, the location
of a thrilling scene from the 1966 film ...

8. [The Batman (2022) Locations](https://www.latlong.net/location/the-batman-2022-locations-427)
Source: Latitude and Longitude Finder

The Batman (2022) was filmed in Liverpool and London, England, UK. Filming also took place at Cardington Studios 
and Warner Bros. Leavesden Studios.

9. [Where The Batman Filmed (All Gotham Real-Life 
Locations)](https://screenrant.com/where-was-the-batman-movie-filmed/)
Date published: Mar 4, 2022
Source: Screen Rant

Gotham Square - Wellington Square, Liverpool, England​​ The entirety of Wellington Square in Liverpool, England was
used as The Batman's filming ...

Out: None

[Step 0: Duration 3.30 seconds| Input tokens: 2,399 | Output tokens: 146]

...

[Step 2: Duration 10.73 seconds| Input tokens: 11,138 | Output tokens: 1,903]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Gotham coordinates                                                                                             
  gotham_coords = (40.7128, -74.0060)                                                                              
                                                                                                                   
  # Predefined coordinates for known locations                                                                     
  coordinates_dict = {                                                                                             
      "St. George's Hall, Liverpool, England": (53.4084, -2.9916),                                                 
      "Two Temple Place, London, England": (51.5128, -0.0918),                                                     
      "Wollaton Hall, Nottingham, England": (52.9522, -1.1497),                                                    
      "Carnegie Mellon University, Pittsburgh, Pennsylvania, USA": (40.4432, -80.0056),                            
      "The Farmiloe Building, Acton Lane, England": (51.5128, -0.2051),                                            
      "Queensboro Bridge, New York, USA": (40.7051, -73.9346),                                                     
      "Wall Street, New York, USA": (40.7069, -74.0113),                                                           
      "Mehrangarh Fort, Jodhpur, India": (26.3044, 73.0243),                                                       
      "Turda, Cluj County, Romania": (46.7833, 23.5833),                                                           
      "Westwood, Los Angeles, California, USA": (34.0736, -118.4004),                                              
      "Cardington Studios, Northamptonshire, England": (52.2555, -0.4353),                                         
      "Warner Bros. Leavesden Studios, Hertfordshire, England": (51.7447, -0.4119),                                
      "Knebworth House, Knebworth, Hertfordshire, England": (51.8814, -0.2903),                                    
      "Wellington Square, Liverpool, England": (53.4084, -2.9916),                                                 
      "Chicago, Illinois, USA": (41.8781, -87.6298),                                                               
      "Lamborghini, Sant'Agata Bolognese, Italy": (44.8406, 11.6184),                                              
      "Ferrari, Maranello, Italy": (44.5403, 10.9921),                                                             
      "Porsche, Leipzig, Germany": (51.3397, 12.3731),                                                             
      "Aston Martin, Gaydon, England": (52.3814, -1.3197),                                                         
      "Koenigsegg, Angered, Sweden": (59.3444, 13.4909),                                                           
      "Pagani, San Cesario sul Panaro, Italy": (44.8333, 10.9833),                                                 
      "McLaren, Woking, England": (51.3778,[38;2;248;248;242;

Execution logs:
Batman filming locations and travel times to Gotham: {'Wellington Square, Liverpool, England': 8.81}
Supercar factories and travel times to Gotham: {"Lamborghini, Sant'Agata Bolognese, Italy": 10.77, 'Ferrari, 
Maranello, Italy': 10.74, 'Porsche, Leipzig, Germany': 10.36, 'Aston Martin, Gaydon, England': 9.01, 'Koenigsegg, 
Angered, Sweden': 9.92, 'Pagani, San Cesario sul Panaro, Italy': 10.71, 'McLaren, Woking, England': 9.13, 'Bugatti,
Molsheim, France': 10.1, 'Rimac, Zagreb, Croatia': 11.13, 'Hypercar, Abu Dhabi, United Arab Emirates': 17.16}

Out - Final answer:                                      Location  Travel Time to Gotham (hours)
0       Wellington Square, Liverpool, England                           8.81
1    Lamborghini, Sant'Agata Bolognese, Italy                          10.77
2                   Ferrari, Maranello, Italy                          10.74
3                   Porsche, Leipzig, Germany                          10.36
4               Aston Martin, Gaydon, England                           9.01
5                 Koenigsegg, Angered, Sweden                           9.92
6       Pagani, San Cesario sul Panaro, Italy                          10.71
7                    McLaren, Woking, England                           9.13
8                   Bugatti, Molsheim, France                          10.10
9                      Rimac, Zagreb, Croatia                          11.13
10  Hypercar, Abu Dhabi, United Arab Emirates                          17.16

[Step 3: Duration 19.54 seconds| Input tokens: 18,141 | Output tokens: 3,360]

In [ ]:
# !pip install pandas==0.22

In [8]:
result

,Location,Travel Time to Gotham (hours)
0,"Wellington Square, Liverpool, England",8.81
1,"Lamborghini, Sant'Agata Bolognese, Italy",10.77
2,"Ferrari, Maranello, Italy",10.74
3,"Porsche, Leipzig, Germany",10.36
4,"Aston Martin, Gaydon, England",9.01
5,"Koenigsegg, Angered, Sweden",9.92
6,"Pagani, San Cesario sul Panaro, Italy",10.71
7,"McLaren, Woking, England",9.13
8,"Bugatti, Molsheim, France",10.10
9,"Rimac, Zagreb, Croatia",11.13


We could already improve this a bit by throwing in some dedicated planning steps, and adding more prompting.

In [9]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
                ```
                1. Perform a web search to find a comprehensive list of Batman filming locations worldwide.
2. For each filming location, visit the source URL to confirm the coordinates.
3. Calculate the cargo plane transfer time from each filming location to Gotham.
4. Perform a web search to find a comprehensive list of supercar factories worldwide.
5. For each supercar factory, visit the source URL to confirm the coordinates.
6. Calculate the cargo plane transfer time from each supercar factory to Gotham.
7. Filter both lists to include only locations with the same cargo plane transfer time.
8. Organize the filtered locations and their details into a pandas dataframe.
9. Provide the final answer as the pandas dataframe.


                ```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  Thought: I will perform a web search to find a comprehensive list of Batman filming locations worldwide.         
  Code:                                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Batman filming locations and travel times to Gotham: {'Wellington Square, Liverpool, England': 8.81}
Supercar factories and travel times to Gotham: {"Lamborghini, Sant'Agata Bolognese, Italy": 10.77, 'Ferrari, 
Maranello, Italy': 10.74, 'Porsche, Leipzig, Germany': 10.36, 'Aston Martin, Gaydon, England': 9.01, 'Koenigsegg, 
Angered, Sweden': 9.92, 'Pagani, San Cesario sul Panaro, Italy': 10.71, 'McLaren, Woking, England': 9.13, 'Bugatti,
Molsheim, France': 10.1, 'Rimac, Zagreb, Croatia': 11.13, 'Hypercar, Abu Dhabi, United Arab Emirates': 17.16}

...

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_location_coords = {}                                                                                      
                                                                                                                   
  for location in batman_filming_locations:                                                                        
      search_query = f"{location} coordinates"                                                                     
      result = web_search(query=search_query)                                                                      
      batman_location_coords[location] = result                                                                    
                                                                                                                   
  print(batman_location_coords)                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
{"St George's Hall, Liverpool, England": '## Search Results\n0. [St George\'s Hall, 
Liverpool](https://en.wikipedia.org/wiki/St_George%27s_Hall,_Liverpool)\nSource: Wikipedia\n\nSt George\'s Place, 
Liverpool, England · 53°24′31″N 2°58′48″W\ufeff / \ufeff53.4086°N 2.9801°W\ufeff / 53.4086; -2.9801 · SJ 349 907 · 
1841–1854.\n\n1. [Where is Liverpool, the UK on Map Lat Long 
Coordinates](https://www.latlong.net/place/liverpool-the-uk-28017.html)\nSource: Latitude and Longitude 
Finder\n\nThe latitude of Liverpool, the UK is 53.400002, and the longitude is -2.983333. Liverpool, the UK is 
located at United Kingdom country in the Cities place ...\n\n2. [File:St Georges Hall, Liverpool, from the 
southwest.jpg](https://en.m.wikipedia.org/wiki/File:St_Georges_Hall,_Liverpool,_from_the_southwest.jpg)\nSource: 
Wikipedia\n\nHeading=56.25° Kartographer map based on OpenStreetMap. View this and other nearby images on: 
OpenStreetMap · info 53.407860; -2.981265 ...\n\n3. [St George\'s 
Hall](https://www.atlasobscura.com/places/st-georges-hall)\nDate published: Sep 21, 2018\nSource: Atlas 
Obscura\n\nSt George\'s Hall. St George\'s Place Liverpool, England United Kingdom. Copy Address. 53.408419, 
-2.980235. Copy Coordinates. Visit Website. Open ...\n\n4. [St George\'s Hall, Non Civil Parish - 
1361677](https://historicengland.org.uk/listing/the-list/list-entry/1361677)\nSource: Historic England\n\nSt 
George\'s Hall. St George\'s Hall, St George\'s Place, Liverpool, L1 1JJ. Listed on the National Heritage List for 
England.\n\n5. [File:St George\'s Hall, Liverpool - geograph.org.uk - 
4855.jpg](https://commons.wikimedia.org/wiki/File:St_George%27s_Hall,_Liverpool_-_geograph.org.uk_-_4855.jpg)\nDate
published: Jan 21, 2025\nSource: Wikipedia\n\nKartographer map based on OpenStreetMap. View this and other nearby 
images on: OpenStreetMap · info 53.407000; -2.990000. Licensing. edit ...\n\n6. [St. Georges Hall Lime Street, 
Liverpool L1 1JJ England](https://www.tripexpress.org/attraction-st-george-s-hall-415398/)\nSource: 
tripexpress.org\n\nSt. Georges Hall Lime Street, Liverpool L1 1JJ England. Get Direction. Latitude: 53° 24\' 
30.132" N Longitude: 2° 58\' 45.696" W. Time zone: Europe/London. 4.5. + ...\n\n7. [Discover - Welcome to St 
George\'s Hall, Liverpool](https://stgeorgeshallliverpool.co.uk/discover/)\nSource: St George\'s Hall\n\nSituated 
opposite Lime Street station, St George\'s Hall forms an intrinsic part of Liverpool\'s William Brown Conservation 
Area and provides a magnificent ...\n\n8. [St George\'s Hall (1841-54), Lime Street, Liverpool, by 
...](https://victorianweb.org/art/architecture/elmes/1.html)\nDate published: Sep 14, 2009\nSource: The Victorian 
Web\n\nSt George\'s Hall is on St George\'s Plateau opposite the large sandstone Gothic structure which fronts 
Liverpool\'s Lime Street Station.\n\n9. [St Georges Hall| Destination 
Merseyside](https://www.merseyrail.org/destination-merseyside/view-all-locations/st-georges-hall/)\nSource: 
Merseyrail\n\nFind out how to get to St Georges Hall with Merseyrail. Timetables and ticket prices also 
available.', 'Anfield Cemetery, Liverpool, England': '## Search Results\n0. [Anfield 
Cemetery](https://en.wikipedia.org/wiki/Anfield_Cemetery)\nSource: Wikipedia\n\nAnfield Cemetery ; August 1863 · 
Anfield, Liverpool, Merseyside · England · 53°26′19″N 2°57′30″W\ufeff / \ufeff53.4387°N 2.9584°W\ufeff / 53.4387; 
-2.9584.\n\n1. [GPS coordinates of Anfield Cemetery, United Kingdom. 
...](https://latitude.to/articles-by-country/gb/united-kingdom/61028/anfield-cemetery)\nSource: 
Latitude.to\n\nSatellite map of Anfield Cemetery. Anfield Cemetery, or the City of Liverpool Cemetery, is located 
in Anfield, a district of Liverpool, Merseyside, England.\n\n2. [Anfield Cemetery and Crematorium - 
Liverpool](https://www.findagrave.com/cemetery/638757/anfield-cemetery-and-crematorium)\nSource: Find a 
Grave\n\nGet directions 238 Priory Road Anfield, Metropolitan Borough o

[Step 7: Duration 48.90 seconds| Input tokens: 142,561 | Output tokens: 2,704]

[Step 8: Duration 1.18 seconds| Input tokens: 170,889 | Output tokens: 2,890]

HfHubHTTPError: 422 Client Error: Unprocessable Entity for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: Root=1-67bf4a96-261cdcff37858cd701943daf;a075ac58-6715-4bbf-a6ac-df2e5b76a86f)

{'message': 'Input validation error: `inputs` tokens + `max_new_tokens` must be <= 32769. Given: 36401 `inputs` tokens and 2048 `max_new_tokens`', 'type': 'invalid_request_error', 'param': None, 'code': None}
{
  "id": "918189cdfad63b8c",
  "error": {
    "message": "Input validation error: `inputs` tokens + `max_new_tokens` must be <= 32769. Given: 36401 `inputs` tokens and 2048 `max_new_tokens`",
    "type": "invalid_request_error",
    "param": null,
    "code": null
  }
}


Thanks to these quick changes, we obtained a much more concise report by simply providing our agent a detailed prompt, and giving it planning capabilities!

💸 But as you can see, the context window is quickly filling up. So **if we ask our agent to combine the results of detailed search with another, it will be slower and quickly ramp up tokens and costs**.

➡️ We need to improve the structure of our system.

## ✌️ Splitting the task between two agents

Multi-agent structures allow to separate memories between different sub-tasks, with two great benefits:
- Each agent is more focused on its core task, thus more performant
- Separating memories reduces the count of input tokens at each step, thus reducing latency and cost.

Let's create a team with a dedicated web search agent, managed by another agent.

The manager agent should have plotting capabilities to redact its final report: so let us give it access to additional imports, including `matplotlib`, and `geopandas` + `shapely` for spatial plotting.

In [30]:
model = HfApiModel(
    "Qwen/Qwen2.5-Coder-32B-Instruct", provider="together", max_tokens=8096
)

web_agent = CodeAgent(
    model=model,
    tools=[
        GoogleSearchTool(),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],
    name="web_agent",
    description="Browses the web to find information",
    verbosity_level=0,
    max_steps=10,
)

The manager agent will need to do some mental heavy lifting.

So we give it the stronger model [DeepSeek-R1](https://huggingface.co/deepseek-ai/DeepSeek-R1), and add a `planning_interval` to the mix.

In [31]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [32]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    final_answer
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=HfApiModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

In [33]:
manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | Qwen/Qwen2.5-Coder-32B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a google web search for  │ query (`string`): The search      │
            │                             │ your query then returns a string  │ query to perform.                 │
            │                             │ of the top search results.        │ filter_year (`integer`):          │
            │                             │                                   │ Optionally restrict results to a  │
            │                             │                                   │ certain year                      │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.     

In [34]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ HfApiModel - deepseek-ai/DeepSeek-R1 ──────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
                ```
                <think>
Okay, let's start breaking down the task. The user wants a map with Batman filming locations and supercar 
factories, each with their cargo plane transfer times to Gotham. The final map should have at least six points.

First, I need to get the list of Batman filming locations. Since I don't have that data, I'll need to ask the 
web_agent to find them. Once I have the locations, I'll need their coordinates. Maybe the web_agent can also get 
those, or I might need to use a geocoding tool, but the available tools don't include that. Wait, the tools only 
have calculate_cargo_travel_time and final_answer. So the web_agent will have to handle both finding the locations 
and their coordinates.

Next, for each Batman location, calculate the travel time from Gotham using the calculate_cargo_travel_time tool. 
That requires the origin (Gotham's coordinates) and destination coordinates, plus the cruising speed. The default 
speed is 750 km/h, so I can use that unless specified otherwise.

Then, I need to find supercar factories. Again, the web_agent has to search for these. Once I have the factories' 
names and locations, get their coordinates. Then calculate their travel times to Gotham as well.

The task says to include supercar factories with the same cargo plane transfer time as the Batman locations. So I 
need to match the times. For example, if a Batman location has a 5-hour transfer time, find factories that also 
take 5 hours. But if there aren't enough matches, maybe include some factories with close times to reach at least 
six points total. However, the user specifies "at least 6 points in total," so combining both Batman and factories,
even if their times aren't identical, as long as the total is six or more. Wait, the original task says "some 
supercar factories with the same cargo plane transfer time. You need at least 6 points in total." So maybe the 
factories must have the same time as some of the Batman locations. For example, if there are three Batman 
locations, find three factories with matching times. But if that's not possible, perhaps include both Batman and 
factories regardless, as long as the total is six.

But the exact requirement is a bit ambiguous. The plan should focus on steps to gather both sets, calculate their 
times, and ensure the total is six. So proceed as:

1. Get Batman locations and their coordinates.
2. Calculate their travel times.
3. Get supercar factories and their coordinates.
4. Calculate their travel times.
5. Combine both sets, ensuring at least six points. If necessary, include factories even if their times don't 
match, but the user might prefer matching. However, the task says "some supercar factories with the same cargo 
plane transfer time," implying that at least some factories should match the Batman times. So perhaps select 
factories whose times match any of the Batman times. If not enough, maybe proceed with what's available but note 
the discrepancy.

Once all data is collected, create a DataFrame with columns for lat, lon, name, and travel time. Use the example 
code to plot, color by travel time, and save the image.

So the plan steps:

1. Use web_agent to find Batman filming locations and their coordinates.
2. For each Batman location, calculate travel time from Gotham.
3. Use web_agent to find supercar factories and their coordinates.
4. For each factory, calculate travel time from Gotham.
5. Select factories with travel times matching any Batman location's time to ensure at least six total points.
6. Combine Batman and factory data into a dataset.
7. Plot the map with color based on travel time and save to saved_map.png.
</think>

1. Task the web agent to compile a list of all Batman filming locations worldwide, including their geographic 
coordinat

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
 [CODE STEPS]:                                                                                                     
Here are the code steps to solve the problem:                                                                      
                ```                                                                                                
                <think>                                                                                            
Okay, let's start coding step by step. First, I need to get the Batman filming locations. Since I can't directly   
access the web, I'll use the web_agent to find them. I'll start by assigning the task to the web_agent to search   
for Batman filming locations with their coordinates.                                                               
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
batman_locations = web_agent(task="Find a list of all Batman movie filming locations worldwide, including their    
latitude and longitude coordinates. Provide the result as a list of dictionaries with keys 'name', 'latitude',     
'longitude'.")                                                                                                     
print("Batman Filming Locations:", batman_locations)                                                               
```<end_code>                                                                                                      
                                                                                                                   

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  <think>                                                                                                          
  Okay, let's start coding step by step. First, I need to get the Batman filming locations. Since I can't          
  directly access the web, I'll use the web_agent to find them. I'll start by assigning the task to the web_agent  
  to search for Batman filming locations with their coordinates.                                                   
                                                                                                                   
  Code:                                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 10 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID:
Root=1-67bf5c26-293c91d00e62b19e53c12498;6e58ffbe-1346-4da8-ab99-e5e971cfed9f)

You have exceeded your monthly included credits for Inference Providers. Pay-as-you-go above your included PRO 
quota will be available soon.

[Step 9: Duration 0.70 seconds| Input tokens: 73,008 | Output tokens: 5,233]

[Step 10: Duration 0.29 seconds| Input tokens: 83,705 | Output tokens: 5,950]

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/together/v1/chat/completions (Request ID: Root=1-67bf5c26-48c397de2a7950f652fc45a2;19b5b847-fcb8-43f9-9415-bb7063260890)

You have exceeded your monthly included credits for Inference Providers. Pay-as-you-go above your included PRO quota will be available soon.

In [17]:
manager_agent.python_executor.state["fig"]

KeyError: 'fig'